In [1]:
# Analytic fields mapping from Transactions
#   - Ariba-Ond_Buyer-dev, Ariba-Ond_S4-dev
#   - BuyerPOLoad.xml (Full Search: "ariba.analytics.fact.SSPPOLineItem", File Path: xml)
#   - BuyerReceiptLoad.xml (Full Search: "ariba.analytics.fact.Receipt", File Path: xml)
#
# Step 1:
#   - Main factor is to bid for required quantity  
#       - Bid details (Bid Id, Item Id, Requested Qty)
#       - Item details (Id, Name, Description)
#       - Supplier details (Supplier Id, Agreed qty, Quoted price)
#       - API call to get JSON and convert to CSV
#       - EDS Bids: ItemId = Item Id, SubmitForId = Supplier Id
#           - Qty: "itemTermUniqueName": "QUANTITY", "itemTermType": "QUANTITY", "quantityValue": 3
#           - Price: "itemTermUniqueName": "PRICE", "itemTermType": "MONEY", "moneyValue": 65.66, "attribute": "USD"
#       - EDS Items Paginated Request: Get commodity code from Item Id (commodityCodeId = Commodity ID)
#  
# Q: How to identify PR's/PO's created from which bidding event ??
#  
# Either through "contracts" Or "quick sourcing" workflow we can link awarded supplier
#     
# Step 2: Get related PO's/Line items for the awarded bid items
#   - Extract PO, Line Items and Receipt details
# 
# Step 3: Compute supplier characteristics --> Supplier's characteristic computation is based on previously supplied items
#   - Map supplier/PO/Item level details with all the Receipts 
#   - Compute Quality
#       - Compute quality at PO Vs Receipts items.. How many received against rejected ?
#           - If rejections compute : POLineItem.java, ReceiptItem.java
#               - Option1: Sort by Receipts, Go to the latest Receipts and get Total Rejected/Total Received to check all received
#               - Option2: Go thru all GRs check if any rejections in each GR.
#       - Quality for an supplier at an item level
#   - Compute Timeliness
#       - Receipt date against NeedbyDate wanted
#           - For all PO check if NeedByDate, Order confirmation, Shipping confirmation available : Ignore if 01/01/1970
#                If available : Compute does all items received within asked NeedByDate
#       - Supplier timeliness at an item level
#
# Step 4: Build UI by proposing weightage to parameters (Price, Quality, Timeliness)
#
# Step 5: Build prediction model to predict supplier characteristics for a given supplier/price (Computed at PO's/Reciept level)
#    - Model on historical data: Supplier, Item ID, Price, Quality, Timeliness --> Consider Single item
#    - Target Variables: Predict Supplier charactersitics (Quality, Timeliness) for a given item/price
#    - User Input: Given Item Id, % Weightage (Price, Quality, Timeliness)
#    - Output: Predict supplier matching the criteria by evaluating LP file.
# 
# Step 6: Compute LP expressions/evaluation
#    - Consider Price, Quality, Timeliness user's % weightage input
#    - LP evaluation based on % weightage input.. by default equal weightage for all suppler characteristics
#
# Step 7: Get supplier risk details from Supplier Risk API (Stretch goal)
#
# Step 8: Provide explanation for the desired supplier to award.

In [49]:
'''
System: gcpdev02
Realm: rptRealm54
Inspector: Downloaded 5000 records matching PO's 
'''
import pandas as pd
import csv
import numpy as np

order = pd.read_csv('./input/PO_Details.csv')
order.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   POId                    120 non-null    object 
 1   OrderID                 120 non-null    object 
 2   POName                  120 non-null    object 
 3   Amount                  120 non-null    object 
 4   DeliveryTime            120 non-null    float64
 5   ReceiptTime             120 non-null    float64
 6   InvoiceTime             120 non-null    float64
 7   OnTimeOrLate            88 non-null     object 
 8   RejectedItems           120 non-null    object 
 9   SubstitutedItems        120 non-null    object 
 10  OrderedDate             120 non-null    object 
 11  NeedByDate              120 non-null    object 
 12  SupplierId              120 non-null    object 
 13  SupplierName            44 non-null     object 
 14  SupplierPublicName      0 non-null      fl

In [50]:
order.columns

Index(['POId', 'OrderID', 'POName', 'Amount', 'DeliveryTime', 'ReceiptTime',
       'InvoiceTime', 'OnTimeOrLate', 'RejectedItems', 'SubstitutedItems',
       'OrderedDate', 'NeedByDate', 'SupplierId', 'SupplierName',
       'SupplierPublicName', 'SupplierCity', 'SupplierState',
       'SupplierCountry', 'SupplierLocationId', 'POLineNumber', 'Title',
       'Description', 'LineSupplierId', 'LineSupplierName',
       'LineSupplierPublicName', 'LineSupplierCity', 'LineSupplierState',
       'LineSupplierCountry', 'LineSupplierLocationId', 'Orderd_Quantity',
       'LineNeedByDate', 'Amount.1', 'LineItemCount', 'POCount',
       'OrderedDate.1', 'CommodityId', 'CommodityName', 'UNSPSCCodeId',
       'SupplierPartNumber', 'PartName', 'OrderType'],
      dtype='object')

In [51]:
order_details = order[['POId', 'POName', 'SupplierId', 'SupplierName', 'POLineNumber', 'CommodityId', 'CommodityName', 'LineNeedByDate', 'Orderd_Quantity', 
                       'Amount', 'OrderedDate', 'OrderType']]

In [52]:
order_details

,POId,POName,SupplierId,SupplierName,POLineNumber,CommodityId,CommodityName,LineNeedByDate,Orderd_Quantity,Amount,OrderedDate,OrderType
0,PO11,SSP_Reports-Inv777169670,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,1.0,109.95000,9/21/2011 12:00 AM,Direct
1,PO33,Copy of S e s,0000000100,JCN Technologies (AN - Fulfillment FT),1,82111502,NaN,1/1/1970 12:00 AM,1.0,220.02000,5/7/2022 12:00 AM,Direct
2,PO33,Copy of S e s,0000000100,JCN Technologies (AN - Fulfillment FT),2,82111502,NaN,1/1/1970 12:00 AM,1.0,220.02000,5/7/2022 12:00 AM,Direct
3,PO35,material catalog+adhoc service,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,1.0,10.25000,5/7/2022 12:00 AM,Direct
4,PO38,Copy of Latha HF Verification -1,0000000100,JCN Technologies (AN - Fulfillment FT),100001,432118,NaN,1/1/1970 12:00 AM,0.0,10.89063,8/17/2022 12:00 AM,Direct
...,...,...,...,...,...,...,...,...,...,...,...,...
115,PO68,Ml49,SUPPLIERRPC05,NaN,100001,432118,NaN,11/1/2022 12:00 AM,0.0,"1,577.91875",11/1/2022 12:00 AM,Direct
116,PO69,ML51,SUPPLIERRPC05,NaN,100002,432118,NaN,11/1/2022 12:00 AM,0.0,"6,674.73125",11/1/2022 12:00 AM,Direct
117,PO69,ML51,SUPPLIERRPC05,NaN,100001,432118,NaN,11/1/2022 12:00 AM,0.0,"6,674.73125",11/1/2022 12:00 AM,Direct
118,PO69,ML51,SUPPLIERRPC05,NaN,2,432118,NaN,11/1/2022 12:00 AM,250.0,"6,674.73125",11/1/2022 12:00 AM,Direct


In [53]:
receipt = pd.read_csv('./input/Receipt_Details.csv')
receipt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   OrderId                   175 non-null    object 
 1   ReceiptId                 175 non-null    object 
 2   ReceiptDate               175 non-null    object 
 3   LineItemNumber            175 non-null    int64  
 4   Receipt_Quantity          175 non-null    float64
 5   LineType                  175 non-null    object 
 6   DateOfDelivery            175 non-null    object 
 7   NumberPreviouslyAccepted  175 non-null    float64
 8   NumberAccepted            175 non-null    float64
 9   NumberPreviouslyRejected  175 non-null    object 
 10  NumberRejected            175 non-null    object 
dtypes: float64(3), int64(1), object(7)
memory usage: 15.2+ KB


In [54]:
receipt['NumberPreviouslyAccepted'].dtype

dtype('float64')

In [73]:
from numpy import float64


if (isinstance(receipt['NumberPreviouslyAccepted'], str) == True):
    receipt['NumberPreviouslyAccepted'] = receipt['NumberPreviouslyAccepted'].str.replace(',','').astype(np.float64)

if (isinstance(receipt['NumberAccepted'], str) == True):
    receipt['NumberAccepted'] = receipt['NumberAccepted'].str.replace(',','').astype(np.float64)

if (isinstance(receipt['NumberPreviouslyRejected'], object) == True):
    receipt['NumberPreviouslyRejected'] = receipt['NumberPreviouslyRejected'].str.replace(',','').astype(np.float64)

if (isinstance(receipt['NumberRejected'], object) == True):
    receipt['NumberRejected'] = receipt['NumberRejected'].str.replace(',','').astype(np.float64)

receipt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   OrderId                   175 non-null    object 
 1   ReceiptId                 175 non-null    object 
 2   ReceiptDate               175 non-null    object 
 3   LineItemNumber            175 non-null    int64  
 4   Receipt_Quantity          175 non-null    float64
 5   LineType                  175 non-null    object 
 6   DateOfDelivery            175 non-null    object 
 7   NumberPreviouslyAccepted  175 non-null    float64
 8   NumberAccepted            175 non-null    float64
 9   NumberPreviouslyRejected  175 non-null    float64
 10  NumberRejected            175 non-null    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 15.2+ KB


In [74]:
receipt.columns

Index(['OrderId', 'ReceiptId', 'ReceiptDate', 'LineItemNumber',
       'Receipt_Quantity', 'LineType', 'DateOfDelivery',
       'NumberPreviouslyAccepted', 'NumberAccepted',
       'NumberPreviouslyRejected', 'NumberRejected'],
      dtype='object')

In [75]:
receipt

,OrderId,ReceiptId,ReceiptDate,LineItemNumber,Receipt_Quantity,LineType,DateOfDelivery,NumberPreviouslyAccepted,NumberAccepted,NumberPreviouslyRejected,NumberRejected
0,PO33,RC52,5/7/2022 12:00 AM,2,1.0,NonCatalog,1/1/1970 12:00 AM,0.0,1.0,0.0,0.0
1,PO33,SS11,1/1/1970 12:00 AM,1,1.0,NonCatalog,1/1/1970 12:00 AM,0.0,0.0,0.0,0.0
2,PO35,RC53,5/7/2022 12:00 AM,1,1.0,Catalog,1/1/1970 12:00 AM,0.0,1.0,0.0,0.0
3,PO37,RC55,6/9/2022 12:00 AM,1,10.0,Catalog,1/1/1970 12:00 AM,0.0,2.0,0.0,0.0
4,PO38,RC57,8/17/2022 12:00 AM,1,1.0,Catalog,1/1/1970 12:00 AM,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
170,PO69,RC148,11/1/2022 12:00 AM,1,180.0,Catalog,1/1/1970 12:00 AM,27.0,33.0,35.0,0.0
171,PO69,RC149,11/1/2022 12:00 AM,1,180.0,Catalog,1/1/1970 12:00 AM,60.0,0.0,35.0,21.0
172,PO69,RC149,11/1/2022 12:00 AM,2,250.0,Catalog,1/1/1970 12:00 AM,225.0,0.0,60.0,45.0
173,PO69,RC150,11/1/2022 12:00 AM,2,250.0,Catalog,1/1/1970 12:00 AM,225.0,25.0,105.0,0.0


In [76]:
order_receipts = pd.merge(order_details, receipt,  how='inner', left_on=['POId','POLineNumber'], right_on = ['OrderId','LineItemNumber'])
order_receipts

,POId,POName,SupplierId,SupplierName,POLineNumber,CommodityId,CommodityName,LineNeedByDate,Orderd_Quantity,Amount,...,ReceiptId,ReceiptDate,LineItemNumber,Receipt_Quantity,LineType,DateOfDelivery,NumberPreviouslyAccepted,NumberAccepted,NumberPreviouslyRejected,NumberRejected
0,PO33,Copy of S e s,0000000100,JCN Technologies (AN - Fulfillment FT),1,82111502,NaN,1/1/1970 12:00 AM,1.0,220.02000,...,SS11,1/1/1970 12:00 AM,1,1.0,NonCatalog,1/1/1970 12:00 AM,0.0,0.0,0.0,0.0
1,PO33,Copy of S e s,0000000100,JCN Technologies (AN - Fulfillment FT),2,82111502,NaN,1/1/1970 12:00 AM,1.0,220.02000,...,RC52,5/7/2022 12:00 AM,2,1.0,NonCatalog,1/1/1970 12:00 AM,0.0,1.0,0.0,0.0
2,PO35,material catalog+adhoc service,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,1.0,10.25000,...,RC53,5/7/2022 12:00 AM,1,1.0,Catalog,1/1/1970 12:00 AM,0.0,1.0,0.0,0.0
3,PO38,Copy of Latha HF Verification -1,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,1.0,10.89063,...,RC57,8/17/2022 12:00 AM,1,1.0,Catalog,1/1/1970 12:00 AM,0.0,1.0,0.0,0.0
4,PO39,TestMl,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,10.0,108.90625,...,RC58,10/26/2022 12:00 AM,1,10.0,Catalog,1/1/1970 12:00 AM,0.0,5.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,PO69,ML51,SUPPLIERRPC05,NaN,2,432118,NaN,11/1/2022 12:00 AM,250.0,"6,674.73125",...,RC150,11/1/2022 12:00 AM,2,250.0,Catalog,1/1/1970 12:00 AM,225.0,25.0,105.0,0.0
170,PO69,ML51,SUPPLIERRPC05,NaN,1,432118,NaN,11/1/2022 12:00 AM,180.0,"6,674.73125",...,RC147,11/1/2022 12:00 AM,1,180.0,Catalog,1/1/1970 12:00 AM,0.0,27.0,0.0,35.0
171,PO69,ML51,SUPPLIERRPC05,NaN,1,432118,NaN,11/1/2022 12:00 AM,180.0,"6,674.73125",...,RC148,11/1/2022 12:00 AM,1,180.0,Catalog,1/1/1970 12:00 AM,27.0,33.0,35.0,0.0
172,PO69,ML51,SUPPLIERRPC05,NaN,1,432118,NaN,11/1/2022 12:00 AM,180.0,"6,674.73125",...,RC149,11/1/2022 12:00 AM,1,180.0,Catalog,1/1/1970 12:00 AM,60.0,0.0,35.0,21.0


In [77]:
order_receipts.columns

Index(['POId', 'POName', 'SupplierId', 'SupplierName', 'POLineNumber',
       'CommodityId', 'CommodityName', 'LineNeedByDate', 'Orderd_Quantity',
       'Amount', 'OrderedDate', 'OrderType', 'OrderId', 'ReceiptId',
       'ReceiptDate', 'LineItemNumber', 'Receipt_Quantity', 'LineType',
       'DateOfDelivery', 'NumberPreviouslyAccepted', 'NumberAccepted',
       'NumberPreviouslyRejected', 'NumberRejected'],
      dtype='object')

In [78]:
order_receipts_details = order_receipts[['POId', 'POName', 'SupplierId', 'SupplierName', 'POLineNumber','CommodityId', 'CommodityName', 
                                         'LineNeedByDate', 'Orderd_Quantity', 'Amount', 'OrderedDate', 'OrderType', 'ReceiptId','ReceiptDate', 
                                         'DateOfDelivery', 'LineType', 'Receipt_Quantity', 'NumberPreviouslyAccepted', 'NumberAccepted',
                                         'NumberPreviouslyRejected', 'NumberRejected']]
order_receipts_details

,POId,POName,SupplierId,SupplierName,POLineNumber,CommodityId,CommodityName,LineNeedByDate,Orderd_Quantity,Amount,...,OrderType,ReceiptId,ReceiptDate,DateOfDelivery,LineType,Receipt_Quantity,NumberPreviouslyAccepted,NumberAccepted,NumberPreviouslyRejected,NumberRejected
0,PO33,Copy of S e s,0000000100,JCN Technologies (AN - Fulfillment FT),1,82111502,NaN,1/1/1970 12:00 AM,1.0,220.02000,...,Direct,SS11,1/1/1970 12:00 AM,1/1/1970 12:00 AM,NonCatalog,1.0,0.0,0.0,0.0,0.0
1,PO33,Copy of S e s,0000000100,JCN Technologies (AN - Fulfillment FT),2,82111502,NaN,1/1/1970 12:00 AM,1.0,220.02000,...,Direct,RC52,5/7/2022 12:00 AM,1/1/1970 12:00 AM,NonCatalog,1.0,0.0,1.0,0.0,0.0
2,PO35,material catalog+adhoc service,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,1.0,10.25000,...,Direct,RC53,5/7/2022 12:00 AM,1/1/1970 12:00 AM,Catalog,1.0,0.0,1.0,0.0,0.0
3,PO38,Copy of Latha HF Verification -1,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,1.0,10.89063,...,Direct,RC57,8/17/2022 12:00 AM,1/1/1970 12:00 AM,Catalog,1.0,0.0,1.0,0.0,0.0
4,PO39,TestMl,0000000100,JCN Technologies (AN - Fulfillment FT),1,432118,NaN,1/1/1970 12:00 AM,10.0,108.90625,...,Direct,RC58,10/26/2022 12:00 AM,1/1/1970 12:00 AM,Catalog,10.0,0.0,5.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,PO69,ML51,SUPPLIERRPC05,NaN,2,432118,NaN,11/1/2022 12:00 AM,250.0,"6,674.73125",...,Direct,RC150,11/1/2022 12:00 AM,1/1/1970 12:00 AM,Catalog,250.0,225.0,25.0,105.0,0.0
170,PO69,ML51,SUPPLIERRPC05,NaN,1,432118,NaN,11/1/2022 12:00 AM,180.0,"6,674.73125",...,Direct,RC147,11/1/2022 12:00 AM,1/1/1970 12:00 AM,Catalog,180.0,0.0,27.0,0.0,35.0
171,PO69,ML51,SUPPLIERRPC05,NaN,1,432118,NaN,11/1/2022 12:00 AM,180.0,"6,674.73125",...,Direct,RC148,11/1/2022 12:00 AM,1/1/1970 12:00 AM,Catalog,180.0,27.0,33.0,35.0,0.0
172,PO69,ML51,SUPPLIERRPC05,NaN,1,432118,NaN,11/1/2022 12:00 AM,180.0,"6,674.73125",...,Direct,RC149,11/1/2022 12:00 AM,1/1/1970 12:00 AM,Catalog,180.0,60.0,0.0,35.0,21.0


In [79]:
allCommodity = order_receipts_details['CommodityName'].unique()
allCommodity

array([nan])

In [80]:
'''
Ariba-Ond_Buyer-dev/receiving/ariba/receiving/core/ReceiptItem.java

Total Number Accepted = Number Previously Accepted + Number accepted
Total Number Rejected = Number Previously Rejected + Number Rejected
Total Quantity Received = Total Number Accepted  + Total Number Rejected

Total Amount Accepted = Amount Previously Accepted + Amount Accepted
Total Amount Rejected = Amount Previously Rejected + Amount Rejected
Total Amount Received = Total Amount Accepted  + Total Amount Rejected
'''

order_receipts_details['TotalNumberAccepted'] = order_receipts_details['NumberPreviouslyAccepted'] + order_receipts_details['NumberAccepted']
order_receipts_details['TotalNumberRejected'] = order_receipts_details['NumberPreviouslyRejected'] + order_receipts_details['NumberRejected']
order_receipts_details['TotalQuantityReceived'] = order_receipts_details['TotalNumberAccepted'] + order_receipts_details['TotalNumberRejected']
order_receipts_details.to_csv('./output/order_receipts.csv', mode='w', header=True, encoding='utf-8', index=False)

/var/folders/t4/204xh5g57vv3c7mjndmh4y7r0000gn/T/ipykernel_2738/1122676370.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_receipts_details['TotalNumberAccepted'] = order_receipts_details['NumberPreviouslyAccepted'] + order_receipts_details['NumberAccepted']
